## Exercise 3 - Language Identification -Reloaded and Convoluted

In [20]:

#Importing the tools
import csv
import re
import numpy as np
import pandas as pd



Loading the data

In [21]:
url_train_dev = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTOZ2rC82rhNsJduoyKYTsVeH6ukd7Bpxvxn_afOibn3R-eadZGXu82eCU9IRpl4CK_gefEGsYrA_oM/pub?gid=1863430984&single=true&output=tsv'
url_test = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vT-KNR9nuYatLkSbzSRgpz6Ku1n4TN4w6kKmFLkA6QJHTfQzmX0puBsLF7PAAQJQAxUpgruDd_RRgK7/pub?gid=417546901&single=true&output=tsv'

Splitting the data into tweet and labels

In [22]:

from io import StringIO
import requests

def load_dataset(url):
    r = requests.get(url)
    data = r.content.decode('utf8')
    df = pd.read_csv(StringIO(data), sep='\t')
    df.columns = ['tweet', 'label']
    return df
    

In [23]:
df_train  = load_dataset(url_train_dev)
df_test = load_dataset(url_test)
print(df_train[0:10])


                                               tweet label
0  يا من أناديها ويخنقني البكاء  ويكاد صمت الدمع ...    ar
1  فيه فرق بين اهل غزة اللى مطحونين من ناحيتين وب...    ar
2  ﻋﻦ ﺍﻟﻠﺤﻈﺔ اﻟﺤﻠﻮﺓﺓ ﺍﻟﻠﻲ ﺑﺘﻐﻤﺾ ﻓﻴﻬﺎ ﻋﻴﻨﻴﻚ ﺑﺘﻔﻜﺮ ...    ar
3                                  يا ابو سلو عرفتني    ar
4  ب50 ريال أكفل معتمر في رمضان ، ولك بإذن الله م...    ar
5  توجيه كيفية تثبيت البرامج الثابتة ROM التحميل ...    ar
6  {وأنه هو أغنى وأقنى} [النجم:48] http://t.co/is...    ar
7  اللهم قدر لنا الفرح بكل اشكاله ، انت الكريم ال...    ar
8  #غزه_تحت_القصف  داعش أخواني حيل عندكم بالمدنيي...    ar
9  {يعلمون ظاهرا من الحياة الدنيا وهم عن الآخرة ه...    ar


Data Preprocessing

In [24]:
#Converting all the tweets to lower string
df_train["tweet"] = df_train["tweet"].str.lower()
filter_dt=df_train.groupby('label').count()['tweet']

#Removing all tweets with less than 15 samples to train from 
filter_dt=filter_dt.loc[filter_dt.values>15].index
df_train=df_train[df_train['label'].isin(filter_dt)]
df_train.reset_index(inplace=True)
filter_dt=df_train.groupby('label').count()['tweet']

#Removing all labels from test which are not there in training
df_test=df_test[df_test['label'].isin(df_train.label.unique())]
df_test.reset_index(inplace=True)

In [25]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52460 entries, 0 to 52459
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   52460 non-null  int64 
 1   tweet   52460 non-null  object
 2   label   52460 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB


In [26]:
df_train.head()

,index,tweet,label
0,0,يا من أناديها ويخنقني البكاء ويكاد صمت الدمع ...,ar
1,1,فيه فرق بين اهل غزة اللى مطحونين من ناحيتين وب...,ar
2,2,ﻋﻦ ﺍﻟﻠﺤﻈﺔ اﻟﺤﻠﻮﺓﺓ ﺍﻟﻠﻲ ﺑﺘﻐﻤﺾ ﻓﻴﻬﺎ ﻋﻴﻨﻴﻚ ﺑﺘﻔﻜﺮ ...,ar
3,3,يا ابو سلو عرفتني,ar
4,4,ب50 ريال أكفل معتمر في رمضان ، ولك بإذن الله م...,ar


In [27]:
df_train.label.unique()

array(['ar', 'ca', 'de', 'el', 'en', 'es', 'fa', 'fr', 'he', 'hi', 'id',
       'it', 'ja', 'ko', 'ms', 'nl', 'pl', 'pt', 'ru', 'sr', 'sv', 'th',
       'tl', 'tr', 'uk', 'und', 'vi', 'zh-CN'], dtype=object)

Retrieving the tweets and labels from the training and the test set.

In [28]:
X_train = df_train.tweet
y_train = df_train.label
X_test = df_test.tweet
y_test = df_test.label

print('Training tweet shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test tweet shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Training tweet shape:  (52460,)
Training labels shape:  (52460,)
Test tweet shape:  (13217,)
Test labels shape:  (13217,)


Data Cleaning - removing all the unnecessary data from the text

In [29]:
def preprocessor(text):
  
  text = re.sub(r"http\S+", "", text)       ##Removing URL
  text = re.sub('@[^\s]+','',text)          ##Removing @Username
  text = re.sub('[\W]+', ' ', text.lower()) ##Removing NonCharacter eg.emojis
  text = re.sub('<[^>]*>', '', text)        ##Removing entire HTML
  text = re.sub(r'[0-9]+', '', text)        ## Removing numbers
  
  return text

#Applying the text cleaning to the train data
X_train = X_train.apply(preprocessor)
X_train[0:30] #Checking the data 

0     يا من أناديها ويخنقني البكاء ويكاد صمت الدمع أ...
1     فيه فرق بين اهل غزة اللى مطحونين من ناحيتين وب...
2     ﻋﻦ ﺍﻟﻠﺤﻈﺔ اﻟﺤﻠﻮﺓﺓ ﺍﻟﻠﻲ ﺑﺘﻐﻤﺾ ﻓﻴﻬﺎ ﻋﻴﻨﻴﻚ ﺑﺘﻔﻜﺮ ...
3                                     يا ابو سلو عرفتني
4     ب ريال أكفل معتمر في رمضان ولك بإذن الله مثل أ...
5     توجيه كيفية تثبيت البرامج الثابتة rom التحميل ...
6                            وأنه هو أغنى وأقنى النجم  
7     اللهم قدر لنا الفرح بكل اشكاله انت الكريم الذي...
8      غزه_تحت_القصف داعش أخواني حيل عندكم بالمدنيين...
9      يعلمون ظاهرا من الحياة الدنيا وهم عن الآخرة ه...
10               افضل كتاب قرأته هو أمي ابراهام لنكولن 
11    ولأن ه م م لائ ك ة ص غار ن ع ش ق ات كاءة رؤوس ...
12        خ لاصة الح ب هي ت فكر بقلبهآ وهو ي فكر بعقله 
13    جميل آن يفهمك من تحبب ويخآف عليك و يغآر عليكك ...
14    حتى الندم على المعصيه تؤجر عليه سبحانك يالله م...
15     اها يا بيبي والله اتهرست علي تويتر و ع الفيس ...
16     لا يقاتلونكم جميعا إلا في قرى محصنة أو من ورا...
17          طبت منك نهائيا بس كلي رجاء ما ترجع م

Checking the shape and assigning new variables to train and test data to be used for further processing of data

In [30]:
print(X_train.shape)
print(y_train.shape)

X_train_n = X_train
y_train_n = y_train
X_test_n = X_test
y_test_n = y_test


(52460,)
(52460,)


Preparing the final data for the model


1.   Tokenizing text data
2.   Encoding labels
3.   Finding the maximum length of the data and vocab size  for the input shape of the model input layer
4.   Printing the details




In [31]:
from keras.preprocessing.text import Tokenizer                    
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout,MaxPooling1D, Conv1D, Flatten , Embedding
from keras.preprocessing import text, sequence
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from keras import utils

#Calulating the length of the doc to be fed into the input layer of the model
def maxilen(lines):
	return max([len(s.split()) for s in lines])

#Fitting the dataTokenizer
def Generatetokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

#Creating the tokenizer
tokenizer = Generatetokenizer(X_train_n[0:])

#Encoding Text
def text_encoder(tokenizer, lines, length):
	
	encoded = tokenizer.texts_to_sequences(lines)
	padded = pad_sequences(encoded, maxlen=length, padding='post')
	return padded

#Calculating the maximum document length for the input shape for the model input
length = maxilen(X_train_n[0:])

#Calculating the vocabulary size
vocab_size = len(tokenizer.word_index) + 1

print('Maximum document length: %d' % length)
print('Vocabulary size of the data: %d' % vocab_size)

#Encoding the data
trainX = text_encoder(tokenizer, X_train_n[0:], length)
testX = text_encoder(tokenizer, X_test_n[0:], length)
print(trainX.shape, testX.shape)

encoder = LabelEncoder()
encoder.fit(y_train_n)
y_trainning_labeled = encoder.transform(y_train_n)
y_testing_labeled = encoder.transform(y_test_n)

num_classes = np.max(y_trainning_labeled) + 1
y_trainning_labeled = utils.to_categorical(y_trainning_labeled, num_classes)
y_testing_labeled = utils.to_categorical(y_testing_labeled, num_classes)


Maximum document length: 1224
Vocabulary size of the data: 102072
(52460, 1224) (13217, 1224)


In [47]:
#Splitting the cleaned data into train and validation 90/10 ratio 
 
from sklearn.model_selection import train_test_split
X_train_n , X_val , y_train_n , y_val = train_test_split (trainX, y_trainning_labeled, test_size= 0.1,random_state=42)

Creating model using Keras and implementing different layers 


Model Combination 1

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=length))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [69]:
from tensorflow import keras
from tensorflow.keras import layers

In [78]:
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
history = model.fit(X_train_n, y_train_n,
                    epochs=5,batch_size=50)  

#save the model
model.save('model.h5')

Epoch 1/5
945/945 [==============================] - 104s 110ms/step - loss: 0.4073 - accuracy: 0.9759
Epoch 2/5
945/945 [==============================] - 104s 110ms/step - loss: 0.3662 - accuracy: 0.9798
Epoch 3/5
945/945 [==============================] - 104s 110ms/step - loss: 0.3962 - accuracy: 0.9816
Epoch 4/5
945/945 [==============================] - 103s 109ms/step - loss: 0.4450 - accuracy: 0.9825
Epoch 5/5
945/945 [==============================] - 102s 108ms/step - loss: 0.4137 - accuracy: 0.9836


In [81]:
score = model.evaluate(X_val, y_val,
                       batch_size=32, verbose=1)
print('Validation accuracy :', score[1])

164/164 [==============================] - 1s 6ms/step - loss: 10.3832 - accuracy: 0.8111
Validation accuracy : 0.8110941648483276


In [82]:
#Prediction on Test data
score = model.evaluate(testX, y_testing_labeled)
print('Test accuracy :', score[1])


414/414 [==============================] - 3s 6ms/step - loss: 19.5548 - accuracy: 0.6781
Test accuracy : 0.6780661344528198


Model Combination 2

In [98]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=length))
model.add(Conv1D(filters=128, kernel_size=3,strides=2 ,activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(num_classes))
model.add(Activation('softmax'))


from tensorflow.keras.optimizers import Adamax

opt = keras.optimizers.Adamax(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
history = model.fit(X_train_n, y_train_n,
                    epochs=5,batch_size=50) 


Epoch 1/5
945/945 [==============================] - 51s 54ms/step - loss: 1.6020 - accuracy: 0.5264
Epoch 2/5
945/945 [==============================] - 51s 54ms/step - loss: 1.2532 - accuracy: 0.6405
Epoch 3/5
945/945 [==============================] - 51s 54ms/step - loss: 1.0993 - accuracy: 0.6903
Epoch 4/5
945/945 [==============================] - 51s 54ms/step - loss: 0.9515 - accuracy: 0.7337
Epoch 5/5
945/945 [==============================] - 51s 54ms/step - loss: 0.8383 - accuracy: 0.7795


In [99]:
score = model.evaluate(X_val, y_val,
                       batch_size=32, verbose=1)
print('Validation accuracy :', score[1])



164/164 [==============================] - 1s 6ms/step - loss: 0.8567 - accuracy: 0.7735
Validation accuracy : 0.7735417485237122


In [100]:
#Prediction on Test data
score = model.evaluate(testX, y_testing_labeled)
print('Test accuracy :', score[1])

414/414 [==============================] - 2s 6ms/step - loss: 1.0085 - accuracy: 0.6405
Test accuracy : 0.6404630541801453


Model Combination 3

In [111]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=length))
model.add(Conv1D(filters=64, kernel_size=3,strides=2 ,activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(num_classes))
model.add(Activation('softmax'))


from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.001)
model.compile(loss='huber',optimizer=opt, metrics=['accuracy'])
history = model.fit(X_train_n, y_train_n,
                    epochs=5,batch_size=50) 


Epoch 1/5
945/945 [==============================] - 99s 105ms/step - loss: 0.0085 - accuracy: 0.6433
Epoch 2/5
945/945 [==============================] - 101s 106ms/step - loss: 0.0039 - accuracy: 0.8463
Epoch 3/5
945/945 [==============================] - 101s 106ms/step - loss: 0.0025 - accuracy: 0.8997
Epoch 4/5
945/945 [==============================] - 101s 107ms/step - loss: 0.0019 - accuracy: 0.9234
Epoch 5/5
945/945 [==============================] - 102s 108ms/step - loss: 0.0015 - accuracy: 0.9405


In [112]:
score = model.evaluate(X_val, y_val,
                       batch_size=32, verbose=1)
print('Validation accuracy :', score[1])


164/164 [==============================] - 1s 5ms/step - loss: 0.0045 - accuracy: 0.8315
Validation accuracy : 0.8314906358718872


In [113]:
#Prediction on Test data
score = model.evaluate(testX, y_testing_labeled)
print('Test accuracy :', score[1])

414/414 [==============================] - 2s 5ms/step - loss: 0.0080 - accuracy: 0.7067
Test accuracy : 0.7066656351089478


Model Combination 4

In [117]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=length))
model.add(Conv1D(filters=128, kernel_size=3,strides=2 ,activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(num_classes))
model.add(Activation('softmax'))


from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.001)
model.compile(loss='huber',optimizer=opt, metrics=['accuracy'])
history = model.fit(X_train_n, y_train_n,
                    epochs=5,batch_size=50) 


Epoch 1/5
945/945 [==============================] - 102s 108ms/step - loss: 0.0083 - accuracy: 0.6534
Epoch 2/5
945/945 [==============================] - 102s 108ms/step - loss: 0.0036 - accuracy: 0.8546
Epoch 3/5
945/945 [==============================] - 102s 108ms/step - loss: 0.0022 - accuracy: 0.9126
Epoch 4/5
945/945 [==============================] - 103s 109ms/step - loss: 0.0015 - accuracy: 0.9381
Epoch 5/5
945/945 [==============================] - 103s 109ms/step - loss: 0.0012 - accuracy: 0.9545


In [118]:
score = model.evaluate(X_val, y_val,
                       batch_size=32, verbose=1)
print('Validation accuracy :', score[1])


164/164 [==============================] - 1s 6ms/step - loss: 0.0044 - accuracy: 0.8324
Validation accuracy : 0.8324437737464905


In [119]:
#Prediction on Test data
score = model.evaluate(testX, y_testing_labeled)
print('Test accuracy :', score[1])

414/414 [==============================] - 2s 5ms/step - loss: 0.0085 - accuracy: 0.7055
Test accuracy : 0.7055307626724243


In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=length))
model.add(Conv1D(filters=128, kernel_size=3,strides=2 ,activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(num_classes))
model.add(Activation('softmax'))


In [122]:

from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_hinge',optimizer=opt, metrics=['accuracy'])
history = model.fit(X_train_n, y_train_n,
                    epochs=5,batch_size=100) 

Epoch 1/5
473/473 [==============================] - 58s 122ms/step - loss: 0.0594 - accuracy: 0.9690
Epoch 2/5
473/473 [==============================] - 57s 121ms/step - loss: 0.0504 - accuracy: 0.9729
Epoch 3/5
473/473 [==============================] - 57s 121ms/step - loss: 0.0451 - accuracy: 0.9759
Epoch 4/5
473/473 [==============================] - 57s 120ms/step - loss: 0.0417 - accuracy: 0.9773
Epoch 5/5
473/473 [==============================] - 58s 122ms/step - loss: 0.0386 - accuracy: 0.9789


In [123]:
score = model.evaluate(X_val, y_val,
                       batch_size=32, verbose=1)
print('Validation accuracy :', score[1])


164/164 [==============================] - 1s 6ms/step - loss: 0.3926 - accuracy: 0.7791
Validation accuracy : 0.7790697813034058


In [124]:
#Prediction on Test data
score = model.evaluate(testX, y_testing_labeled)
print('Test accuracy :', score[1])

414/414 [==============================] - 2s 5ms/step - loss: 0.5520 - accuracy: 0.7166
Test accuracy : 0.7165771126747131


In [138]:
from sklearn.model_selection import GridSearchCV

para_grid= { 'alpha': [0.1,0.01,0.001,0.001],
        'loss': ['huber','categorical_hinge','categorical_crossentropy'],
        'optimizer':['adam','adammax','sgd'],
        'dropout': [0.5, 0.4, 0.3, 0.2, 0.1, 0],
        'batch_size':[50,75,100]
          
      }
grid = GridSearchCV(estimator=model,  
                    n_jobs=-1, 
                    verbose=1,
                    cv=5,  
                    param_grid=para_grid,
                    scoring='categorical_crossentropy')

grid_result = grid.fit(trainX,y_trainning_labeled )

ValueError: ignored